In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.1 MB/s eta 0:00:00


**Clone repo**

In [8]:
%cd Intelligent-Talent-Acquisition-Assistant

/content/Intelligent-Talent-Acquisition-Assistant


In [9]:
!pip install groq pdfminer.six boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
from pdfminer.high_level import extract_text
import os

def parse_all_resumes(folder_path):
    resumes = {}

    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            file_path = os.path.join(folder_path, file)
            text = extract_text(file_path)
            resumes[file] = text

    return resumes

In [12]:
folder_path = "/content/drive/MyDrive/Gen4AI2/TalentAI_Resumes"
resumes = parse_all_resumes(folder_path)

print(resumes.keys())

dict_keys(['resume1.pdf', 'resume10.pdf', 'resume2.pdf', 'resume3.pdf', 'resume4.pdf', 'resume5.pdf', 'resume6.pdf', 'resume7.pdf', 'resume8.pdf', 'resume9.pdf'])


In [14]:
import os
from google.colab import userdata
GROQ_KEY=userdata.get('GROQ_KEY')
os.environ["GROQ_API_KEY"] = GROQ_KEY

In [15]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)



---

---





---



In [20]:
!pip install groq pdfminer.six

In [21]:
import os
import json
import re
from groq import Groq
from pdfminer.high_level import extract_text

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
folder_path = "/content/drive/MyDrive/Gen4AI2/TalentAI_Resumes"

In [24]:
def get_resume_text(path):
    text = extract_text(path)

    # Limit size (Groq free tier safe)
    if len(text) > 4000:
        text = text[:4000]

    return text

In [25]:
def clean_resume_text(text):

    # Fix spaced capital letters (C H A R L E S → CHARLES)
    text = re.sub(
        r'\b(?:[A-Z]\s){2,}[A-Z]\b',
        lambda x: x.group(0).replace(" ", ""),
        text
    )

    # Remove non-ascii characters
    text = text.encode("ascii", "ignore").decode()

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [26]:
import os
from google.colab import userdata
GROQ_KEY=userdata.get('GROQ_KEY')
os.environ["GROQ_API_KEY"] = GROQ_KEY

In [27]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [28]:
def parse_resume(text):

    prompt = f"""
You are an AI Resume Parser.

Extract ONLY the following in valid JSON format:

{{
"name": "",
"email": "",
"skills": [],
"education": "",
"experience": "",
"projects": []
}}

Resume:
{text}

Return ONLY JSON.
"""

    try:
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )

        output = response.choices[0].message.content

        if output is None or output.strip() == "":
            return None

        return json.loads(output)

    except Exception as e:
        print("Groq error:", e)
        return None

In [29]:
structured_resumes = {}

for file in os.listdir(folder_path):

    if file.endswith(".pdf"):

        print("Processing:", file)

        path = os.path.join(folder_path, file)

        raw_text = get_resume_text(path)
        cleaned_text = clean_resume_text(raw_text)

        parsed_data = parse_resume(cleaned_text)

        if parsed_data:
            structured_resumes[file] = parsed_data
        else:
            print("Skipped:", file)

Processing: resume1.pdf
Processing: resume10.pdf
Processing: resume2.pdf
Processing: resume3.pdf
Groq error: Expecting value: line 1 column 1 (char 0)
Skipped: resume3.pdf
Processing: resume4.pdf
Processing: resume5.pdf


Processing: resume6.pdf
Processing: resume7.pdf
Processing: resume8.pdf
Processing: resume9.pdf


In [30]:
print(json.dumps(structured_resumes, indent=2))

{
  "resume1.pdf": {
    "name": "Charles McTurland",
    "email": "cmcturland@email.com",
    "skills": [
      "Python (Django)",
      "Javascript (NodeJS ReactJS, jQuery)",
      "SQL (MySQL, PostgreSQL, NoSQL)",
      "HTML5/CSS",
      "AWS",
      "Unix",
      "Git"
    ],
    "education": "B.S. Computer Science, University of Pittsburgh (September 2008 - April 2012)",
    "experience": [
      {
        "title": "Software Engineer",
        "company": "Embark",
        "location": "New York, NY",
        "dates": "January 2015 - current",
        "description": [
          "Worked with product managers to re-architect a multi-page web app into a single page web-app, boosting yearly revenue by $1.4M",
          "Constructed the logic for a streamlined ad-serving platform that scaled to our 35M users, which improved the page speed by 15% after implementation",
          "Tested software for bugs and operating speed, xing bugs and documenting processes to increase efciency by 18%

In [31]:
with open("parsed_resumes.json", "w") as f:
    json.dump(structured_resumes, f, indent=4)

In [32]:
from google.colab import files
files.download("parsed_resumes.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>